In [ ]:
# @title Run MSAffect

# !pip freeze > requirements.txt
# !pip freeze | xargs pip uninstall -y
# !pip install -r requirements.txt

!rm -r results
!python main.py


In [ ]:
# @title Push to MSAffect GitHub

import os

GITHUB_TOKEN = ""  # @param {type: "string"}

!git add .
!git commit -m "Project update from Google Colab" || echo "nothing to commit, working tree clean"

print('Process finished with exit code', os.system(f"git push https://{GITHUB_TOKEN}@github.com/aaravdave/MSAffect.git"))

In [ ]:
# @title Initiate MSAffect Environment

!rm -r sample_data
!git clone https://github.com/aaravdave/MSAffect.git
%cd MSAffect/
!rm -r alphafold
!rm -r colabfold
!git config --global user.email "aarinaarav@gmail.com"
!git config --global user.name "Aarav Dave"

!pip install -q --no-warn-conflicts "colabfold[alphafold-minus-jax] @ git+https://github.com/sokrypton/ColabFold"
!ln -s /usr/local/lib/python3.*/dist-packages/colabfold colabfold
!ln -s /usr/local/lib/python3.*/dist-packages/alphafold alphafold
